In [1]:
import argparse
import sys

import cv2
import numpy as np
from skimage import measure

# Function to detect bright spots (which increase when raining)

In [3]:
def detect_bright_spot(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (11, 11), 0)
    thresh = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)[1]
    # cv2.imshow('mask',thresh)
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=3)
    # cv2.imshow('rain',thresh)
    labels = measure.label(thresh, neighbors=8, background=0)
    mask = np.zeros(thresh.shape, dtype="uint8")
    total = 0
    # loop over the unique components
    for label in np.unique(labels):
        # if this is the background label, ignore it
        if label == 0:
            continue

        # otherwise, construct the label mask and count the
        # number of pixels
        labelMask = np.zeros(thresh.shape, dtype="uint8")
        labelMask[labels == label] = 255
        numPixels = cv2.countNonZero(labelMask)

        # if the number of pixels in the component is sufficiently
        # large, then add it to our mask of "large blobs"
        if numPixels > 1200:
            total += numPixels
            mask = cv2.add(mask, labelMask)
    # cv2.imshow('measured',mask)

    return total

# Function to parse arguments

In [5]:
def check_arg(args=None):
    parser = argparse.ArgumentParser(description='Script for detecting raining condition')
    parser.add_argument('-i', '--input',
                        help='Input video filename',
                        required=True)
    parser.add_argument('-o', '--output',
                        help='Filename for output video',
                        default='output.avi')
    parser.add_argument('-rt', '--rain_threshold',
                        help='Intensity threshold value to be considered raining',
                        default=20000)

    results = parser.parse_args(args)
    return (results.input,
            results.output,
            results.rain_threshold)


# Main function to process video

In [ ]:
# getting variables from sys argument
input, output, rainthres = check_arg(sys.argv[1:])
rainthres = int(rainthres)

In [ ]:
# setting variabes manually
input = 'videos/A0051.mov'
output = 'output/sample_output.avi'
rainthres = 20000

In [8]:
# initiazling cv2 video capture and other variable
cap = cv2.VideoCapture(input)
frame_count = 0
total = 0
per = 1000
flag, frame = cap.read()
assert flag == True
height, width, _ = frame.shape
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
# Define VideoWrite object
# cv2.VideoWrite('arg1',arg2,arg3,(width,heigh))
# arg1:output file name
# arg2:Specify Fourcc code
# arg3: frames per seconds
# FourCC is a 4-byte code used to specify video codec
out = cv2.VideoWriter(output, fourcc, fps, (width, height))
# cap.set(cv2.CAP_PROP_POS_FRAMES, 9000)

while True:

    r, img = cap.read()
    # if there are still frames to process
    if r:
        # detect total bright spot area
        total = detect_bright_spot(img)
        # convert to grayscale and get average intensity variance as indicator for foggyness
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        intensity_variance_per_row = np.var(gray, axis=0)
        avg_variance = np.average(intensity_variance_per_row, axis=0)
        # display information for raining
        if total > rainthres:
            text = 'Raining'
            color = (0, 0, 255)
        else:
            text = 'Not Raining'
            color = (255, 255, 255)

        cv2.putText(img, str(total), (0, 25),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1, (255, 255, 255), 2)
        cv2.putText(img, str(text), (0, 50),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1, color, 2)
        # display information for foggy
        if avg_variance < 3900:
            text = 'Foggy'
            color = (0, 0, 255)
        else:
            text = 'Not Foggy'
            color = (255, 255, 255)
        cv2.putText(img, str(int(avg_variance)), (0, 75),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1, (255, 255, 255), 2)
        cv2.putText(img, str(text), (0, 100),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1, color, 2)

        out.write(img)
        frame_count += 1
        # show frame in cv2
        # cv2.imshow("preview", img)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q'):
            break
    else:
        raise RuntimeError('No more frame')

usage: ipykernel_launcher.py [-h] -i INPUT [-o OUTPUT] [-rt RAIN_THRESHOLD]
ipykernel_launcher.py: error: the following arguments are required: -i/--input


SystemExit: 2

c:\users\zhiqi\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
